In [ ]:
import spacy
import json
import random
import os
import ast
from google.colab import drive
from spacy.training import Example
from spacy.scorer import Scorer
from spacy.tokens import Doc
from spacy import displacy
from spacy.training import Example
from sklearn.metrics import precision_score, recall_score, f1_score, fbeta_score
from sklearn.preprocessing import LabelEncoder
!python -m spacy download es_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 837.5 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
drive.mount('/content/drive')
tfmPath = '/content/drive/MyDrive/TFM'
trainingFilesPath = tfmPath+'/archivos_entrenamiento'
validationFilePath = tfmPath+'/archivos_validacion'
testFilePath = tfmPath+'/archivo_test'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
archivo = trainingFilesPath+'/train.txt'

with open(archivo, 'r', encoding='utf-8') as file:
    content = file.read()
    data = ast.literal_eval(content)


In [ ]:
print(data)

[('Finalmente llegué a Gómez Palacio y visité las instalaciones de Bellas Artes, conocí a los que iban a ser mis alumnos. Temblaba todo el tiempo pese al calor que hacía.', {'entities': [(110, 117, 'gen')]}), ('Cuando quise dejar la fila el japonés se abalanzó sobre mí blandiendo un bastón de madera, el mismo con el que golpeaba a los alumnos que así se lo pedían. Es decir, Ejo ofrecía el bastón, los alumnos decían sí o no y en caso de ser la respuesta afirmativa Ejo les descerrajaba unos planazos que atronaban el espacio en penumbra impregnado de incienso. ', {'entities': [(126, 133, 'gen'), (193, 200, 'gen')]}), ('El que el director encargue a los alumnos más aventajados para que sirvan de monitores para orientar y ayudar a los de menor rendimiento, puede ayudarle y apoyar el trabajo del director.', {'entities': [(34, 41, 'gen')]}), ('De allí que también pueda insertarse, en estas páginas, la evocación de una que otra visita guiada, realizadas expresamente para mis alumnos, cuando as

In [ ]:
archivo_val = validationFilePath+'/val.txt'

with open(archivo_val, 'r', encoding='utf-8') as file:
    content = file.read()
    data_val = ast.literal_eval(content)


In [ ]:
print(data_val)

[('Les corresponde, pues, a maestros y profesores la tarea de organizar las actividades de fomento y disfrute de sus alumnos con la lectura, si bien este compromiso debe ser asumido por toda la comunidad educativa. ', {'entities': [(36, 46, 'profs'), (114, 121, 'alu')]}), ('No es lo mismo partir de cero, con alumnos que no han tenido contacto alguno con libros, que empezar con niños cargados de prejuicios negativos por haber vivido experiencias incómodas en su acercamiento a los libros; o trabajar con otros que aprecian y aman la lectura.', {'entities': [(35, 42, 'alu')]}), ('Por lo demás, esperamos que el tipo de iniciativas y propuestas, a las que vamos a hacer referencia a continuación, vayan depositando en los alumnos unos sedimentos que les permitan construir día a día el hábito lector.', {'entities': [(141, 148, 'alu')]}), ('La biblioteca del centro tiene que ser para los alumnos un espacio acogedor. ', {'entities': [(48, 55, 'alu')]}), ('Este puede ser un momento especial que al

In [ ]:
archivo_test = testFilePath+'/test.txt'

with open(archivo_test, 'r', encoding='utf-8') as file:
    content = file.read()
    data_test = ast.literal_eval(content)

In [ ]:
print(data_test)

[('Las expectativas se han cubierto satisfactoriamente pues todos los alumnos que pasan de la tercera actividad planteada suelen concluir el curso con éxito.', {'entities': [(67, 74, 'alu')]}), ('Y un bajísimo nivel en habilidades instrumentales (lectoescritura y cálculo matemático, básicamente) en los alumnos que superan esa etapa e incluso el bachillerato. Causa del problema: el actual sistema educativo.', {'entities': [(108, 115, 'alu')]}), ('Información Geográfica 11-12 Figura 23. Grupos de docentes de 2011-2, por programa académico, matriculados como alumnos en el Aula Virtual desde febrero de 2011 hasta mayo de 2012.', {'entities': [(112, 119, 'alu')]}), ('Por ello hicimos grupos en os que había unos 3 alumnos de cada centro. Hemos realizado una gymkhana por Cangas de Onis y Covadonga en la que tuvimos que descubrir cosas.', {'entities': []}), ('Los alumnos también tienen una opinión muy positiva de esta herramienta ya que les facilita el seguimiento de la asignatura. Un nivel de

In [ ]:
#Entrenamiento
def entrenar(datos, iteraciones=10):
    datos_gold = datos
    nlp = spacy.blank('es')
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe("ner", last=True)
    for _, anns in datos_gold:
        for ent in anns.get('entities'):
            ner.add_label(ent[2])

    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.select_pipes(disable=other_pipes):
        optimizer = nlp.begin_training()
        for n in range(iteraciones):
            print("Iteración número " + str(n))
            random.shuffle(datos_gold)
            losses = {}
            for texto, anns in datos_gold:
              doc = nlp.make_doc(texto)
              ejemplo = Example.from_dict(doc, anns)
              nlp.update(
                  [ejemplo],
                  drop=0.2,
                  sgd=optimizer,
                  losses=losses)
            print(losses)
    return nlp

In [ ]:
entrenamiento = entrenar(data)

Iteración número 0
{'ner': 1559.2126965897442}
Iteración número 1
{'ner': 789.0536708917493}
Iteración número 2
{'ner': 748.9691840247145}
Iteración número 3
{'ner': 555.8108484579689}
Iteración número 4
{'ner': 496.85190996632747}
Iteración número 5
{'ner': 376.6118056434672}
Iteración número 6
{'ner': 383.32050942086175}
Iteración número 7
{'ner': 326.4692411205806}
Iteración número 8
{'ner': 291.1297424141148}
Iteración número 9
{'ner': 273.6389123128082}


In [ ]:
#Se guarda el modelo
path_modelo = tfmPath+'/modelo'
entrenamiento.to_disk(path_modelo)

In [ ]:
entrenamiento.to_disk("modelo")

In [ ]:
nlp = spacy.load(path_modelo)

/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
#Evaluacion
def my_evaluate(ner_model, examples):
    true_labels = []
    pred_labels = []

    for input_, annotations in examples:
        doc = ner_model(input_)

        true_entities = ["O"] * len(input_)
        for start, end, label in annotations['entities']:
            true_entities[start:end] = [label] * (end - start)

        pred_entities = ["O"] * len(input_)
        for ent in doc.ents:
            pred_entities[ent.start_char:ent.end_char] = [ent.label_] * (ent.end_char - ent.start_char)

        true_labels.extend(true_entities)
        pred_labels.extend(pred_entities)

    precision = precision_score(true_labels, pred_labels, average='weighted', zero_division=1)
    recall = recall_score(true_labels, pred_labels, average='weighted', zero_division=1)
    f1 = f1_score(true_labels, pred_labels, average='weighted', zero_division=1)

    return {
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

results = my_evaluate(nlp, data_val)
print(f"Precisión: {results['precision']}")
print(f"Cobertura (Recall): {results['recall']}")
print(f"F1-score: {results['f1_score']}")


Precisión: 0.9992278887180945
Cobertura (Recall): 0.9339160934228772
F1-score: 0.940498191509132


In [ ]:

# Cargar el modelo entrenado de SpaCy
nlp = spacy.load(path_modelo)  # Usa tu propio modelo entrenado si es necesario


ejemplos_test = []
for texto, anotaciones in data_test:
    doc = nlp.make_doc(texto)
    ejemplo = Example.from_dict(doc, anotaciones)
    ejemplos_test.append(ejemplo)

# Realizar predicciones con el modelo
predicciones = []
etiquetas_reales = []
for texto, anotaciones in data_test:
    # Generar predicciones
    doc = nlp(texto)
    predicciones.append([(ent.text, ent.label_) for ent in doc.ents])

    # Obtener las etiquetas reales del conjunto de datos original
    etiquetas_reales.append([(texto[start:end], label) for start, end, label in anotaciones["entities"]])

print("Predicciones:", predicciones)
print("Etiquetas reales:", etiquetas_reales)

/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Predicciones: [[('alumnos', 'alu')], [('alumnos', 'alu')], [('alumnos', 'alu')], [], [('alumnos', 'alu'), ('alumnos', 'alu')], [('alumnos', 'alu')], [], [], [('alumnos', 'alu')], [('alumnos', 'alu')], [('alumnos', 'alu')], [], [('alumnos', 'alu')], [('alumnos', 'alu')], [], [], [('alumnos', 'alu')], [('alumnos', 'alu'), ('alumnos', 'alu')], [('alumnos', 'alu')], [], [], [('alumnos', 'alu')], [], [('alumnos', 'alu'), ('titulados', 'dis6')], [], [], [('profesores', 'profs')], [], [('profesores', 'profs')], [], [], [('profesores', 'profs')], [], [('profesores', 'profs')], [('profesores', 'profs')], [('profesores', 'profs'), ('alumnos', 'alu')], [('profesores', 'profs')], [('Profesores', 'profs'), ('ingenieros', 'ing')], [('profesores', 'profs'), ('titulados', 'dis6')], [], [('profesores', 'profs')], [('profesores', 'profs')], [], [('profesores', 'profs')], [], [('profesores', 'profs')], [('profesores', 'profs')], [], [('investigadores', 'inv')], [('investigadores', 'inv')], [('investigado

In [ ]:
texto_input = input("Por favor, introduce el texto: ")


Por favor, introduce el texto: En el ámbito de la atención sanitaria, es fundamental brindar apoyo adecuado a quienes enfrentan diversas condiciones de salud. Por ejemplo, el discapacitado puede requerir acceso a terapias de rehabilitación especializadas, mientras que el minusválido a menudo necesita adaptaciones en su entorno para mejorar su movilidad y calidad de vida. De manera similar, el inválido puede beneficiarse de dispositivos de asistencia tecnológica que le permitan realizar tareas cotidianas de forma más independiente.  Existen condiciones más específicas, como la de un tetrapléjico, que generalmente requiere cuidados médicos continuos y soporte especializado para manejar su situación. Además, quienes padecen discapacidad a veces deben enfrentar barreras en el acceso a la educación y el empleo, lo cual limita sus oportunidades de desarrollo personal y profesional.  Por otro lado, algunos individuos que sufren discapacidad debido a accidentes o enfermedades pueden experiment

In [ ]:
doc = nlp(texto_input)
displacy.render(doc, style='ent', jupyter=True)

In [ ]:
if len(doc.ents) == 0:
  print(f"No se han encontrado mejoras según los criterios establecidos.")

for ent in doc.ents:
    if ent.label_ == "gen" and "alumnos" in ent.text:
        print(f"Considere cambiar {ent.text} por 'el alumnado'. ")
    if ent.label_ == "gen" and "profesores" in ent.text:
        print(f"Considere cambiar {ent.text} por 'el profesorado'. ")
    if ent.label_ == "gen" and "investigadores" in ent.text:
        print(f"Considere cambiar {ent.text} por 'el equipo de investigación'. ")
    if ent.label_ == "gen" and "habitantes" in ent.text:
      print(f"Considere cambiar {ent.text} por 'la población'. ")
    if ent.label_ == "gen" and "camareros" in ent.text:
      print(f"Considere cambiar {ent.text} por 'el personal de servicio'. ")
    if ent.label_ == "gen" and "ingenieros" in ent.text:
      print(f"Considere cambiar {ent.text} por 'el equipo de ingeniería'. ")
    if ent.label_ == "gen" and "hombres de negocios" in ent.text:
      print(f"Considere cambiar {ent.text} por 'las personas de negocios'. ")
    if ent.label_ == "gen" and "clientes" in ent.text:
      print(f"Considere cambiar {ent.text} por 'la clientela' ")


    if ent.label_ == "dis" and "discapacitad" in ent.text:
      print(f"Considere cambiar {ent.text} por '(persona) con discapacidad'.")
    if ent.label_ == "dis" and "invalid" in ent.text:
      print(f"Considere cambiar {ent.text} por '(persona) con discapacidad'.")
    if ent.label_ == "dis" and "misunsvalid" in ent.text:
      print(f"Considere cambiar {ent.text} por '(persona) con discapacidad'.")
    if ent.label_ == "dis" and "disminuid" in ent.text:
      print(f"Considere cambiar {ent.text} por '(persona) con discapacidad'.")
    if ent.label_ == "dis" and "retrasad" in ent.text:
      print(f"Considere cambiar {ent.text} por '(persona) con discapacidad'.")
    if ent.label_ == "dis" and "tetrapléjic" in ent.text:
      print(f"Considere cambiar {ent.text} por '(persona) con tetrapléjia'.")
    if ent.label_ == "dis" and "sufr" in ent.text:
      print(f"Considere cambiar {ent.text} por 'tener discapacidad'.")
    if ent.label_ == "dis" and "padec" in ent.text:
      print(f"Considere cambiar {ent.text} por 'tener discapacidad'.")

Considere cambiar discapacitado por '(persona) con discapacidad'.
Considere cambiar tetrapléjico por '(persona) con tetrapléjia'.
Considere cambiar padecen discapacidad por 'tener discapacidad'.
Considere cambiar sufren discapacidad por 'tener discapacidad'.
Considere cambiar disminuido por '(persona) con discapacidad'.
Considere cambiar retrasado por '(persona) con discapacidad'.
